<a href="https://colab.research.google.com/github/satogen/SIGNSTE2021-student-cup-private-5th-place-solution/blob/main/data_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Earth Engine Python APIのインポート
import ee
# GEEの認証・初期化
ee.Authenticate()
ee.Initialize()
from google.colab import drive
drive.mount('/content/drive')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=CYNkW2Es5bRbKAlhwbIvaEjJTEMtJ-u8e4wXnePdN1k&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWiWOnx01l1i5_JFhjiV6mRK2ZuaD_fg4neTkGIdwfmYOX_meaPNq3s

Successfully saved authorization token.
Mounted at /content/drive


In [ ]:
!!pip install geopandas

In [109]:
import os
import  json

import numpy as np

import geopandas as gpd
from PIL import Image
import matplotlib.pyplot as plt

In [110]:
class CFG:
  POINT_GJSON = "/content/drive/MyDrive/卒業研究/data/polygonsToDriveExample.geojson"

  # 保存情報
  SAVE_DRIVE_PATH = '/content/drive/MyDrive/卒業研究/data/nighttimeData'
  SAVE_DRIVE_FOLDER = 'nighttimeData'
  SCALE = 100

  # 対象面積の測定
  AREA = 50000 * 50000

  # 衛星情報
  SATELLITE = 'NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG'
  BAND_NAME = 'avg_rad'
  FROM_DATE='2017-05-01'
  TO_DATE='2017-05-31'


In [101]:
class DataExporter():
  """
  Google Earth Engineからデータをダウンロードする

  Attributes
  ----------
  dataset : ee.ImageCollection
    対象のデータセット
  dataset_image : ee.image.Image
    ダウンロード対象の画像
  satellite : str
    ダウンロード対象の衛星
  from_date : str
    測定開始日
  """
  def __init__(self, satellite=CFG.SATELLITE, from_date=CFG.FROM_DATE, to_date=CFG.TO_DATE, band_name=CFG.BAND_NAME):
    self.dataset = ''
    self.dataset_image = ''

    self.satellite = satellite
    self.band_name = band_name
    self.from_date = from_date
    self.to_date = to_date

    self.read_point()

  def select_dataset(self):
    """
    データセットの取得をする
    """
    self.dataset = ee.ImageCollection(self.satellite).filter(ee.Filter.date(self.from_date, self.to_date))

  def select_band(self):
    """
    対象データセットのバンドの指定と対象データセットの指定をする
    """
    self.dataset_image = dataset.select(self.band_name).median()

  def select(self):
    """
    画像とバンドの取得を同時にする
    """
    self.select_dataset()
    self.select_band()

  def save_image(self, file_name, polygon, save_folder=CFG.SAVE_DRIVE_FOLDER, scale=CFG.SCALE):
    """
    画像の保存をする

    Parameters
    ----------
    file_name : str
        ファイルの名前
    polygon : ee.geometry.Geometry
        ポリゴンデータ
    save_path : str
        保存対象のパス
    scale : int
        画像のスケール
    """
    # Gdriveへ保存
    task = ee.batch.Export.image.toDrive(**{
        'image': self.dataset_image,
        'description': file_name,
        'folder':save_folder, # 
        'scale': scale,
        'region': polygon
    })

    # 処理の実行
    task.start()

  def read_point(self, point_json_path=CFG.POINT_GJSON):
    """
    ポイントの読み込み
    """
    self.point_src = gpd.read_file(point_json_path)
    self.point_features = json.loads(self.point_src.to_json())['features']

  def get_point(self, index):
    """
    Featuresから緯度経度を取得
    """
    return ee.Geometry.Point(self.point_features[index]['geometry']['coordinates'])

  def get_polygon(self, index, area=CFG.AREA):
    """
    ポイントデータからAreaに沿うように、ポリゴンを作成
    """
    return self.point_list[index].buffer(ee.Number(area).sqrt().divide(2), 1).bounds()

  def create_all_polygon(self):
    """
    全てのポリゴンを作成
    """
    self.point_list = [self.get_point(i) for i in range(len(self.point_features))]
    self.polygon_list = [self.get_polygon(i) for i in range(len(self.point_list))]
    return polygon_list

In [102]:
viirs_exporter = DataExporter()
viirs_exporter.select()
polygons = viirs_exporter.create_all_polygon()

In [103]:
polygons = exporter.create_all_polygon()

In [107]:
# 画像の保存
for i in range(len(polygons)):
  viirs_exporter.save_image("export_image_{}".format(i), polygons[i])